Combine embeddings from different integration methods into single adata instance apropritae for scIB (data preparation for integration is in integration_gsPanglaoReactome.ipynb).

In [1]:
import scanpy as sc
import pandas as pd

In [2]:
path_data='/storage/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/qtr/integrated/gsCellType_query/model/'

In [3]:
# Data for integration (raw counts)
adata=sc.read(path_data+'adata_integration_RefQueryTraining.h5ad')

Normalise

In [4]:
# Normalised data
adata_norm=adata.copy()
sc.pp.normalize_total(adata_norm, target_sum=1e6, exclude_highly_expressed=True)
sc.pp.log1p(adata_norm)

Add integrated embeddings

In [9]:
# Integrated embeddings from Sergey's data
adata_int=sc.read(
    '/storage/groups/ml01/workspace/sergei.rybakov/adata_integration_RefQuery.h5ad')
for emb in ['X_scvi','X_seurat','X_symphony']:
    adata_norm.obsm[emb]=adata_int.obsm[emb]

In [10]:
# Integrated embedding from qtr
path_qtr='/storage/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/qtr/integrated/gsCellType_query/model/'
file_qtr='latent_refqueryTraining_alpha_0.1-alpha_kl_60.0-remove_ambient_0-use_hvg_0-batch_study_sample-subset_data_False-data_proportion_1.0-learning_rate_1e-05-hidden_layer_sizes_2048.2048.2048-early_stopping_1-n_hvg_10000-n_epochs_2000.tsv'
adata_norm.obsm['X_qtr']=pd.read_table(path_qtr+file_qtr,index_col=0
                                      ).loc[adata_norm.obs_names,:].values

In [12]:
adata_norm

AnnData object with n_obs × n_vars = 133914 × 6918
    obs: 'leiden_r1_normscl', 'design', 'age', 'sex', 'sample', 'n_counts', 'n_genes', 'mt_frac', 'study', 'study_sample', 'cell_type', 'ref_query'
    var: 'gene_symbol'
    uns: 'log1p'
    obsm: 'X_umap_normscl', 'X_scvi', 'X_seurat', 'X_symphony', 'X_qtr'

Save

In [15]:
# Save for scIB
adata_norm.write(path_data+'adata_norm_integrated_RefQueryTraining.h5ad')